In [12]:
import cudaq
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [15]:
# ----------------------------
# CUDA-Q Kernels
# ----------------------------
@cudaq.kernel
def encode(qubits: cudaq.qview, x: list[float]):
    for i in range(len(x)):
        ry(x[i], qubits[i])


# Define kernel once
@cudaq.kernel
def qattn_kernel(x: list[float], thetas: list[float]):
    q = cudaq.qvector(len(x))
    for i in range(len(x)):
        ry(x[i], q[i])
    for i in range(len(x)-1):
        cx(q[i], q[i+1])
    for i in range(len(x)):
        ry(thetas[i], q[i])

class QuantumAttention(nn.Module):
    def __init__(self, n_qubits=2):
        super().__init__()
        self.n_qubits = n_qubits
        # trainable parameters
        self.params = nn.Parameter(torch.randn(n_qubits))

    def forward(self, x_batch):
        x_np = x_batch.detach().cpu().numpy()
        out_vals = []
        for row in x_np:
            # normalize features to [0, pi]
            x_norm = np.pi * (row[:self.n_qubits] - row.min()) / (row.max() - row.min() + 1e-9)
            result = cudaq.observe(
                qattn_kernel,
                sum([cudaq.spin.z(i) for i in range(self.n_qubits)]),
                list(x_norm),
                self.params.detach().cpu().numpy()
            )
            out_vals.append(result.expectation())
        return torch.tensor(out_vals, dtype=torch.float32, device=x_batch.device).view(-1, 1)
# ----------------------------
# Model
# ----------------------------
class QuantumAttentionRegressor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.attn = QuantumAttention(n_qubits=3)
        self.fc2 = nn.Linear(1, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.attn(x)
        x = self.fc2(x)
        return x


In [17]:
# ----------------------------
# Data
# ----------------------------
dataset_path = "../../data/data.csv"
df = pd.read_csv(dataset_path)

num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

drop_cols = ["ID","Unnamed: 0","Sint","e_Sint"]
features = [c for c in df.columns if c not in drop_cols]

X = df[features].values.astype(np.float32)
y = df["Sint"].values.astype(np.float32).reshape(-1,1)

scalerX, scalery = StandardScaler(), StandardScaler()
X = scalerX.fit_transform(X)
y = scalery.fit_transform(y)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# ----------------------------
# Train
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = QuantumAttentionRegressor(input_dim=X.shape[1]).to(device)
opt = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

for epoch in range(30):
    model.train()
    opt.zero_grad()
    preds = model(X_train.to(device))
    loss = loss_fn(preds, y_train.to(device))
    loss.backward()
    opt.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")


Epoch 1, Loss=1.617873
Epoch 2, Loss=1.616098
Epoch 3, Loss=1.614328
Epoch 4, Loss=1.612563
Epoch 5, Loss=1.610802
Epoch 6, Loss=1.609048
Epoch 7, Loss=1.607298
Epoch 8, Loss=1.605553
Epoch 9, Loss=1.603814
Epoch 10, Loss=1.602081
Epoch 11, Loss=1.600353
Epoch 12, Loss=1.598630
Epoch 13, Loss=1.596913
Epoch 14, Loss=1.595202
Epoch 15, Loss=1.593496
Epoch 16, Loss=1.591796
Epoch 17, Loss=1.590102
Epoch 18, Loss=1.588414
Epoch 19, Loss=1.586732
Epoch 20, Loss=1.585055
Epoch 21, Loss=1.583385
Epoch 22, Loss=1.581720
Epoch 23, Loss=1.580061
Epoch 24, Loss=1.578408
Epoch 25, Loss=1.576761
Epoch 26, Loss=1.575120
Epoch 27, Loss=1.573485
Epoch 28, Loss=1.571856
Epoch 29, Loss=1.570233
Epoch 30, Loss=1.568616


In [18]:
# ----------------------------
# Evaluate
# ----------------------------
model.eval()
with torch.no_grad():
    y_pred = model(X_test.to(device)).cpu().numpy()
    y_true = y_test.cpu().numpy()

mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_true, y_pred)

print("\n--- Test Metrics ---")
print(f"MSE  : {mse:.4f}")
print(f"MAE  : {mae:.4f}")
print(f"RMSE : {rmse:.4f}")
print(f"R²   : {r2:.4f}")


--- Test Metrics ---
MSE  : 1.6545
MAE  : 0.7469
RMSE : 1.2863
R²   : -0.5828
